In [1]:
import torch

In [2]:
from typing import *
from numpy import ndarray
from numpy.typing import NDArray
from torch import Tensor
from xai import Device
from xai.network import Network
import torch
import math

f = Network("cuda", 2).linear(10).relu().linear(1)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
f(torch.tensor([[1.0,2.0]]))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 2x1)